# Chapter 5: Brute-Force Algorithm Validation

## Introduction
In this chapter, we will subject our signature algorithm to a rigorous validation test. The objective is to verify if it can correctly discriminate between all graph structures for a given order `n` by checking it against established mathematical results.

### The Fundamental Hypothesis

A canonical signature algorithm is correct if and only if it meets two fundamental conditions:
1.  **Consistency**: It must assign the same signature to two graphs that are structurally identical (*isomorphic*).
2.  **Uniqueness**: It must assign different signatures to two graphs that are structurally different (*non-isomorphic*).

### Testing Methodology

Rather than starting with a pre-filtered list of unique graphs, we will adopt a "brute-force" approach that tests both conditions simultaneously. This method is more comprehensive as it makes no initial assumptions about the graphs.

The process is as follows:
1.  For a given order `n`, we determine the number of possible edges, `k = n * (n - 1) / 2`.
2.  We generate all **`2^k`** possible labeled graphs. This set contains every possible combination of edges between the `n` vertices.
3.  For each of these graphs, we calculate its canonical signature using our algorithm.
4.  We use a `set` to count the number of unique signatures obtained at the end of the process.

### Validation Criterion

The test is considered a **success** if the number of unique signatures we discover is exactly equal to the known number of non-isomorphic graphs for order `n`. This reference value is provided by the **OEIS A000088** sequence (On-Line Encyclopedia of Integer Sequences).

### Scope of the Experiment and Computational Cost

This method is exhaustive, but its computational cost increases exponentially. We will therefore begin by validating our algorithm for orders `n` from 3 to 6. The table below illustrates the explosion in the number of cases to be processed.

| Order (n) | Possible Edges (k) | Graphs to Test (2^k)  | Expected Unique Graphs (OEIS A000088) |
| :-------- | :------------------- | :---------------------- | :-------------------------------------- |
| 3         | 3                    | 8                       | 4                                       |
| 4         | 6                    | 64                      | 11                                      |
| 5         | 10                   | 1,024                   | 34                                      |
| 6         | 15                   | 32,768                  | 156                                     |
| 7* | 21                   | 2,097,152               | 1,044                                   |
| 8* | 28                   | 268,435,456             | 12,005                                  |
| 9* | 36                   | 68,719,476,736          | 274,668                                 |
| 10*| 45                   | ~3.5 x 10^13            | 12,005,168                              |

*Orders marked with an asterisk require considerable processing time.*



## 5.1 Order 3

In [1]:
from collections import defaultdict
from utils import get_signature, check

# The 8 possible labeled graphs of order 3
graphs_order_3_g6 = [
    'B?',  # 0 edges (1 graph)
    'BQ', 'BS', 'Bw',  # 1 edge (3 graphs)
    'BR', 'Bi', 'Bq',  # 2 edges (3 graphs)
    'B~'   # 3 edges (1 graph)
]

# Dictionary to store the results: {signature: [list_of_g6_strings]}
signature_groups = defaultdict(list)

print(f"Processing {len(graphs_order_3_g6)} graphs of order 3...")

# Loop through all 8 graphs
for g6 in graphs_order_3_g6:
    gs = get_signature(g6)
    compact_signature = gs.sig()
    signature_groups[compact_signature].append(g6)
    print(f"g6:{g6} sig:{compact_signature}")

# --- Analysis of Results ---

num_unique_found = len(signature_groups)
num_expected = 4  # From OEIS A000088 for n=3

print("--- Results for n=3 ---")

check(
    f"We expect {num_expected} unique signatures and got {num_unique_found}", num_unique_found == num_expected)

Processing 8 graphs of order 3...
g6:B? sig:[{nc:0},{nc:0},{nc:0}]
g6:BQ sig:[{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:0,fi:2,rs:1}]
g6:BS sig:[{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:0,fi:2,rs:1}]
g6:Bw sig:[{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]},{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]}]},{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]},{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]}]},{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]},{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]}]}]
g6:BR sig:[{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:0,fi:2,rs:1}]
g6:Bi sig:[{nc:2,fi:0,rs:1},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:1,n:[{nc:2,fi:0,rs:1}]}]
g6:Bq sig:[{nc:2,fi:0,rs:1},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:1,n:[{nc:2,fi:0,rs:1}]}]
g6:B~ sig:[{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:2,n:[{nc:2,ll:2},{nc:2,ll:3}]}]},{

## 5.2 Order 4

In [2]:
import itertools
import networkx as nx
from utils import get_signature, check


def test_order(order_to_test: int, num_expected: int, print_groups: bool = False, check_isomorphic_collisions = False):
    nodes_n = list(range(order_to_test))
    possible_edges = list(itertools.combinations(nodes_n, 2))
    signature_groups = {}
    graphs_generated_count = 0
    total_graphs_to_generate = 2**len(possible_edges)

    non_isomorphic_collisions = []

    modulo = 2500

    while (total_graphs_to_generate / modulo) > 100:
        modulo *= 2

    print(f"--- Testing Order n={order_to_test} ---")
    print(f"Generating and processing {total_graphs_to_generate} graphs...")

    for num_edges_to_select in range(len(possible_edges) + 1):
        for edge_combo in itertools.combinations(possible_edges, num_edges_to_select):
            current_nx_graph = nx.Graph()
            current_nx_graph.add_nodes_from(nodes_n)
            current_nx_graph.add_edges_from(list(edge_combo))
            graphs_generated_count += 1

            gs = get_signature(current_nx_graph)
            compact_signature_for_graph = gs.sig()

            current_g6 = nx.to_graph6_bytes(
                current_nx_graph, header=False).decode('ascii').strip()

            if compact_signature_for_graph not in signature_groups:
                signature_groups[compact_signature_for_graph] = {
                    "first_graph_obj": current_nx_graph.copy(),
                    "g6_examples": [current_g6],
                    "count": 1
                }
            else:
                group_info = signature_groups[compact_signature_for_graph]
                group_info["count"] += 1
                if len(group_info["g6_examples"]) < 5:
                    group_info["g6_examples"].append(current_g6)

                first_graph_in_group = group_info["first_graph_obj"]
                if check_isomorphic_collisions and not nx.is_isomorphic(current_nx_graph, first_graph_in_group):
                    collision_info = {
                        "signature": compact_signature_for_graph,
                        "graph1_g6": group_info["g6_examples"][0],
                        "graph2_g6": current_g6,
                    }
                    non_isomorphic_collisions.append(collision_info)
                    print(
                        f"‼️ COLLISION DETECTED: Signature '{compact_signature_for_graph}'")
                    print(f"   Graph 1 (g6): {collision_info['graph1_g6']}")
                    print(f"   Graph 2 (g6): {collision_info['graph2_g6']}")
                    print(
                        f"   These graphs are NOT isomorphic but got the same signature.")

            # Réintégration de votre affichage de progression (corrigé)
            if graphs_generated_count % modulo == 0 and total_graphs_to_generate > 0:
                percentage_processed = (
                    graphs_generated_count * 100) // total_graphs_to_generate
                current_unique_signatures = len(signature_groups)
                print(
                    f"[{percentage_processed}%] Found {current_unique_signatures} unique signatures out of {num_expected} expected")

    num_unique_found = len(signature_groups)

    print(f"\nTotal graphs generated and tested: {graphs_generated_count}")
    print(f"--- Results for n={order_to_test} ---")

    check(
        f"For n={order_to_test}, we expect {num_expected} unique signatures and got {num_unique_found}",
        num_unique_found == num_expected
    )

    if non_isomorphic_collisions:
        print(
            f"❌ CRITICAL: Found {len(non_isomorphic_collisions)} instance(s) of non-isomorphic graphs sharing a signature:")
        for collision in non_isomorphic_collisions:
            print(f"  Signature: {collision['signature']}")
            print(f"    Graph 1 g6: {collision['graph1_g6']}")
            print(f"    Graph 2 g6: {collision['graph2_g6']}")

    if print_groups:
        print(f"--- Signature Groups (n={order_to_test}) ---")
        for i, (sig_str, group_data) in enumerate(sorted(signature_groups.items())):
            count = group_data["count"]
            example_g6s = group_data["g6_examples"]
            print(
                f"Signature Group {i+1} (Count: {count}): {sig_str} Examples: {example_g6s[:min(3, len(example_g6s))]}..."
            )


test_order(4, num_expected=11, print_groups=True)

--- Testing Order n=4 ---
Generating and processing 64 graphs...

Total graphs generated and tested: 64
--- Results for n=4 ---
✅ For n=4, we expect 11 unique signatures and got 11
--- Signature Groups (n=4) ---
Signature Group 1 (Count: 1): [{nc:0},{nc:0},{nc:0},{nc:0}] Examples: ['C?']...
Signature Group 2 (Count: 6): [{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:0},{nc:0}] Examples: ['C_', 'CO', 'CC']...
Signature Group 3 (Count: 3): [{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]}] Examples: ['C`', 'CQ', 'CK']...
Signature Group 4 (Count: 12): [{nc:2,fi:0,rs:1},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:0,fi:3,rs:1}] Examples: ['Co', 'Cc', 'Cg']...
Signature Group 5 (Count: 12): [{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:1,n:[{nc:2,ll:2}]}]},{nc:1,n:[{nc:2,ll:2}]}]},{nc:2,n:[{nc:2,n:[{nc:2,ll:2},{nc:1,n:[{nc:2,ll:2}]}]},{nc:1,n:[{nc:2,ll:2}]}]},{nc:1,n:[{nc:2,

## 5.3 Order 5

In [3]:
test_order(5, num_expected=34, print_groups=True) # type: ignore

--- Testing Order n=5 ---
Generating and processing 1024 graphs...

Total graphs generated and tested: 1024
--- Results for n=5 ---
❌ For n=5, we expect 34 unique signatures and got 37
--- Signature Groups (n=5) ---
Signature Group 1 (Count: 1): [{nc:0},{nc:0},{nc:0},{nc:0},{nc:0}] Examples: ['D??']...
Signature Group 2 (Count: 10): [{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:0},{nc:0},{nc:0}] Examples: ['D_?', 'DO?', 'DC?']...
Signature Group 3 (Count: 15): [{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:0,fi:4,rs:1}] Examples: ['D`?', 'D_G', 'D_C']...
Signature Group 4 (Count: 30): [{nc:2,fi:0,rs:1},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:0},{nc:0}] Examples: ['Do?', 'Dc?', 'D__']...
Signature Group 5 (Count: 30): [{nc:2,fi:0,rs:1},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:1,n:[{nc:2,fi:0,rs:1}]},{nc:1,n:[{nc:1,n:[{nc:1,ll:2}]}]},{nc:1,n:[{nc:1,n:[{nc:1,

## 5.4 Order 6

In [4]:
test_order(6, num_expected=156, print_groups=False) # type: ignore

--- Testing Order n=6 ---
Generating and processing 32768 graphs...
[7%] Found 33 unique signatures out of 156 expected
[15%] Found 39 unique signatures out of 156 expected
[22%] Found 60 unique signatures out of 156 expected
[30%] Found 67 unique signatures out of 156 expected
[38%] Found 96 unique signatures out of 156 expected
[45%] Found 96 unique signatures out of 156 expected
[53%] Found 134 unique signatures out of 156 expected
[61%] Found 139 unique signatures out of 156 expected
[68%] Found 140 unique signatures out of 156 expected
[76%] Found 180 unique signatures out of 156 expected
[83%] Found 180 unique signatures out of 156 expected
[91%] Found 202 unique signatures out of 156 expected
[99%] Found 219 unique signatures out of 156 expected

Total graphs generated and tested: 32768
--- Results for n=6 ---
❌ For n=6, we expect 156 unique signatures and got 223


## 5.5 Order 7

### 5.5.1 préliminary check 1

In [5]:
import string
import networkx as nx
from graph_signature_v2 import GraphSignatures
from utils import get_signature, check

g1 = nx.from_graph6_bytes("F}_x?".encode('ascii'))
g2 = nx.from_graph6_bytes("F}_pG".encode('ascii'))

mapping = {i: string.ascii_uppercase[i] for i in range(g1.order())}
g1 = nx.relabel_nodes(g1, mapping)
g2 = nx.relabel_nodes(g2, mapping)

print(f"g1 Nodes: {list(g1.nodes())}")
print(f"g2 Nodes: {list(g2.nodes())}")
print(f"g1 Edges: {list(g1.edges())}")
print(f"g2 Edges: {list(g2.edges())}")


sigs1 = GraphSignatures(g1)
sigs2 = GraphSignatures(g2)

pass_number = 1
max_passes = len(g1.nodes()) + 2

print(f"Graph 1: {list(g1.edges())}")
print(f"Graph 2: {list(g2.edges())}")

print(f"Start Graph 1: {sigs1}")
print(f"Start Graph 2: {sigs2}")

while pass_number <= max_passes:
    made_progress1 = sigs1.process_pass(pass_number)
    made_progress2 = sigs2.process_pass(pass_number)
    if sigs1.all_are_finalized() or sigs2.all_are_finalized():
        break
    if not made_progress1 and pass_number > 1:
        if not sigs1.expand_ambiguous_nodes(pass_number):
            break
    if not made_progress2 and pass_number > 1:
        if not sigs2.expand_ambiguous_nodes(pass_number):
            break
    print(f"Pass {pass_number} Graph 1: {sigs1}")
    print(f"Pass {pass_number} Graph 2: {sigs2}")

    pass_number += 1

# --- print Graph ---
print(f"End Graph 1: {sigs1}")
print(f"End Graph 2: {sigs2}")

same_signature = (sigs1.sig() == sigs2.sig())
isomorphic = nx.is_isomorphic(sigs1.graph, sigs2.graph)

check("same signature", same_signature == True)

print("sig1", sigs1)
print("sig2", sigs2)
check("isomorphic", isomorphic == True)

g1 Nodes: ['A', 'B', 'C', 'D', 'E', 'F', 'G']
g2 Nodes: ['A', 'B', 'C', 'D', 'E', 'F', 'G']
g1 Edges: [('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('B', 'C'), ('B', 'D'), ('C', 'F'), ('C', 'G'), ('D', 'F'), ('E', 'F')]
g2 Edges: [('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('B', 'C'), ('B', 'D'), ('C', 'F'), ('C', 'G'), ('D', 'F'), ('F', 'G')]
Graph 1: [('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('B', 'C'), ('B', 'D'), ('C', 'F'), ('C', 'G'), ('D', 'F'), ('E', 'F')]
Graph 2: [('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('B', 'C'), ('B', 'D'), ('C', 'F'), ('C', 'G'), ('D', 'F'), ('F', 'G')]
Start Graph 1: [{label:A,neighbour_count:4},{label:B,neighbour_count:3},{label:C,neighbour_count:4},{label:D,neighbour_count:3},{label:E,neighbour_count:2},{label:F,neighbour_count:3},{label:G,neighbour_count:1}]
Start Graph 2: [{label:A,neighbour_count:4},{label:B,neighbour_count:3},{label:C,neighbour_count:4},{label:D,neighbour_count:3},{label:E,neighbour_count:1},{label:F,neighbou

In [6]:
# test_order(7, num_expected=1044, print_groups=False) # type: ignore